In [ ]:
# Imports & Parameters
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Date range for data
START_DATE = '2020-01-01'
END_DATE = '2026-06-01'

# Top 10 Crypto Tickers by Market Cap
TICKERS = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'BNB-USD', 'SOL-USD', 'TRX-USD', 'ADA-USD', 'DOGE-USD', 'ZEC-USD', 'LINK-USD']

# Strategy parameters
MOMENTUM_WINDOW_MOMENTUM = 20
MOMENTUM_WINDOW_ZSCORE = 60
MOMENTUM_SOFTMAX_BETA = 100
MOMENTUM_SOFTMAX_WEIGHT_CUTTOFF = 0.05
MOMENTUM_EXPOSURE_AMPLIFIER = 3

DSH_DAYS = 20
DSH_NORMALIZATION_GAMMA = 2
DSH_SOFTMAX_BETA = 10
DSH_SOFTMAX_WEIGHT_CUTTOFF = 0
DSH_EXPOSURE_AMPLIFIER = 4

PSMA_WINDOW_SMA = 30
PSMA_WINDOW_VOL = 30
PSMA_SOFTMAX_BETA = 7.5
PSMA_SOFTMAX_WEIGHT_CUTTOFF = 0.05
PSMA_EXPOSURE_AMPLIFIER = 3.43

RRP_WINDOW = 15
RRP_TANH_SCALING = 1
RRP_RETURN_AMPLIFICATION = 3
RRP_SOFTMAX_BETA = 3
RRP_SOFTMAX_WEIGHT_CUTTOFF = 0.05
RRP_EXPOSURE_AMPLIFIER = 2.66

RANGE_WINDOW_RANGE = 20
RANGE_RANGE_AMPLIFIER = 1.5
RANGE_SOFTMAX_BETA = 50
RANGE_SOFTMAX_WEIGHT_CUTTOFF = 0.05
RANGE_EXPOSURE_AMPLIFIER = 3.66

combined_signal_cs = pd.read_pickle("C:\\Users\\blazo\\Documents\\Misc\\QJ\\quant-journey\\research\\crypto_momentum\\combined_signal_cs.pkl")
combined_signal_ts = pd.read_pickle("C:\\Users\\blazo\\Documents\\Misc\\QJ\\quant-journey\\research\\crypto_trend_effects\\combined_signal_ts.pkl")
combined_signal = (combined_signal_cs + combined_signal_ts) / 2

In [ ]:
# Data Downloading and Basic Features

# Download data
data = yf.download(TICKERS, start=START_DATE, end=END_DATE, auto_adjust=True, progress=False)

# Returns
returns = data['Close'].pct_change(fill_method=None)
returns.columns = pd.MultiIndex.from_product([['Returns'], returns.columns])

# Log Returns
log_returns = np.log(data['Close'] / data['Close'].shift(1))
log_returns.columns = pd.MultiIndex.from_product([['LogReturns'], log_returns.columns])

# Next day returns (shifted returns)
next_day_returns = returns.shift(-1)
next_day_returns.columns = pd.MultiIndex.from_product([['NextDayReturns'], returns.columns.get_level_values(1)])

In [ ]:
# Backtest

# Initialize DataFrames to store returns
all_returns = pd.DataFrame()
all_returns_benchmark = pd.DataFrame()

# Iterate through each ticker
for ticker in TICKERS:
    # Prepare data for this ticker
    df_signal = pd.DataFrame({
        'next_day_returns': next_day_returns[('NextDayReturns', ticker)],
        'signal': combined_signal[('CombinedSignal', ticker)]
    }).dropna()
    
    # Strategy: Long-only positions (only when signal > 0)
    df_signal['positions'] = (
        df_signal['next_day_returns'] * 
        df_signal['signal'].clip(lower=0)
    )
    
    # Store strategy returns
    all_returns[ticker] = df_signal['positions']
    
    # Store benchmark returns (buy-and-hold scaled by mean positive signal)
    benchmark_signal = combined_signal[('CombinedSignal', ticker)].clip(lower=0).mean()
    all_returns_benchmark[ticker] = df_signal['next_day_returns'] * benchmark_signal

In [ ]:
# Performance Metrics

def calculate_and_display_metrics(portfolio_returns, cumulative_returns, label="Strategy"):
    """Calculate and display performance metrics for a returns series."""
    total_return = (cumulative_returns.iloc[-1] - 1) * 100
    num_days = len(portfolio_returns)
    num_years = num_days / 365
    annualized_return = (cumulative_returns.iloc[-1] ** (365 / num_days) - 1) * 100
    annualized_volatility = portfolio_returns.std() * np.sqrt(365) * 100
    sharpe_ratio = (portfolio_returns.mean() / portfolio_returns.std()) * np.sqrt(365)
    
    # Calculate maximum drawdown
    cumulative = (1 + portfolio_returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min() * 100
    
    # Display results
    print("=" * 60)
    print(f"BACKTEST RESULTS ({label})")
    print("=" * 60)
    print("Strategy: Momentum")
    print(f"Period: {portfolio_returns.index[0].strftime('%Y-%m')} to {portfolio_returns.index[-1].strftime('%Y-%m')}")
    print(f"Number of years: {num_years:.2f}")
    print("-" * 60)
    print(f"Total Return: {total_return:.2f}%")
    print(f"Annualized Return: {annualized_return:.2f}%")
    print(f"Annualized Volatility: {annualized_volatility:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.3f}")
    print(f"Maximum Drawdown: {max_drawdown:.2f}%")
    print("=" * 60)
    
    return {
        'annualized_return': annualized_return,
        'annualized_volatility': annualized_volatility,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'drawdown': drawdown
    }

# Calculate portfolio returns
portfolio_returns = all_returns.mean(axis=1)
portfolio_returns_benchmark = all_returns_benchmark.mean(axis=1)

# Average exposure of portfolio, but equal to BTC exposure in buy-and-hold benchmark
btc_exposure = combined_signal[('CombinedSignal')].clip(lower=0).mean().mean() 
returns_btc = (next_day_returns[('NextDayReturns', 'BTC-USD')]).dropna() * btc_exposure

# Calculate cumulative returns
cumulative_portfolio_returns = (1 + portfolio_returns).cumprod()
cumulative_portfolio_returns_benchmark = (1 + portfolio_returns_benchmark).cumprod()
cumulative_btc_returns = (1 + returns_btc).cumprod()

# Display strategy metrics
metrics = calculate_and_display_metrics(portfolio_returns, cumulative_portfolio_returns, "Strategy")
annualized_return = metrics['annualized_return']
annualized_volatility = metrics['annualized_volatility']
sharpe_ratio = metrics['sharpe_ratio']
max_drawdown = metrics['max_drawdown']
drawdown = metrics['drawdown']

print("\n")

# Display benchmark metrics
_ = calculate_and_display_metrics(portfolio_returns_benchmark, cumulative_portfolio_returns_benchmark, "Benchmark")

print("\n")

_ = calculate_and_display_metrics(returns_btc, cumulative_btc_returns, "BTC-USD")

In [ ]:
# Visualization

# Create strategy performance and drawdown plots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), gridspec_kw={'height_ratios': [3, 1]})

# Plot 1: Cumulative Returns (Equity Curve)
ax1.plot(cumulative_portfolio_returns.index, cumulative_portfolio_returns, 
         linewidth=2, label='Strategy', color='darkblue')
ax1.plot(cumulative_portfolio_returns_benchmark.index, cumulative_portfolio_returns_benchmark, 
         linewidth=2, label='Benchmark', color='green', alpha=0.7)
ax1.plot(cumulative_btc_returns.index, cumulative_btc_returns, 
         linewidth=2, label='BTC-USD Buy & Hold', color='orange', alpha=0.7)

ax1.set_ylabel('Cumulative Return', fontsize=12)
ax1.set_title('Strategy Performance: Cumulative Returns', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11, loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.axhline(y=1, color='gray', linestyle='--', linewidth=1, alpha=0.5)

# Plot 2: Drawdown
ax2.fill_between(drawdown.index, drawdown * 100, 0, 
                 color='darkred', alpha=0.2, label='Drawdown')
ax2.plot(drawdown.index, drawdown * 100, 
         linewidth=1, color='darkred')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Drawdown (%)', fontsize=12)
ax2.set_title('Strategy Drawdown', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11, loc='lower left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()

plt.show()

In [ ]:
# Rolling Metrics Visualization

# Calculate 1-year rolling metrics for the strategy
rolling_window = 365  # 365 days = 1 year (crypto markets trade daily)

# 1-year rolling return (annualized)
rolling_return = portfolio_returns.rolling(window=rolling_window).apply(
    lambda x: (1 + x).prod() - 1
)

# 1-year rolling volatility (annualized)
rolling_vol = portfolio_returns.rolling(window=rolling_window).std() * np.sqrt(365)

# 1-year rolling Sharpe ratio
rolling_sharpe = (portfolio_returns.rolling(window=rolling_window).mean() / 
                  portfolio_returns.rolling(window=rolling_window).std()) * np.sqrt(365)

# Create the plots
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Plot 1: Rolling Return
ax1 = axes[0]
ax1.plot(rolling_return.index, rolling_return * 100, 
         linewidth=1.5, color='darkblue', label='1-Year Rolling Return')
ax1.axhline(y=annualized_return, color='red', linestyle='--', linewidth=1.5, 
            label=f'Mean: {annualized_return:.2f}%')
ax1.set_ylabel('Return (%)', fontsize=12)
ax1.set_title('1-Year Rolling Annualized Return', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Rolling Volatility
ax2 = axes[1]
ax2.plot(rolling_vol.index, rolling_vol * 100, 
         linewidth=1.5, color='darkgreen', label='1-Year Rolling Volatility')
ax2.axhline(y=annualized_volatility, color='red', linestyle='--', linewidth=1.5, 
            label=f'Mean: {annualized_volatility:.2f}%')
ax2.set_ylabel('Volatility (%)', fontsize=12)
ax2.set_title('1-Year Rolling Annualized Volatility', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# Plot 3: Rolling Sharpe Ratio
ax3 = axes[2]
ax3.plot(rolling_sharpe.index, rolling_sharpe, 
         linewidth=1.5, color='darkorange', label='1-Year Rolling Sharpe Ratio')
ax3.axhline(y=sharpe_ratio, color='red', linestyle='--', linewidth=1.5, 
            label=f'Mean: {sharpe_ratio:.2f}')
ax3.axhline(y=0, color='gray', linestyle='-', linewidth=0.8, alpha=0.5)
ax3.set_xlabel('Date', fontsize=12)
ax3.set_ylabel('Sharpe Ratio', fontsize=12)
ax3.set_title('1-Year Rolling Sharpe Ratio', fontsize=14, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()